In [ ]:
import numpy as np, matplotlib.pyplot as plt, cv2, tqdm
%matplotlib inline

from importlib import reload

from utils import saveVideo, drawShape
import utils
import colorTools

import laneFindingPipeline

In [ ]:
import skvideo.io

videoPrefices = ('project', 'challenge', 'harder_challenge')
allFrames = {}
for videoPrefix in videoPrefices:
    fpath = '%s_video.mp4' % videoPrefix
    reader = skvideo.io.FFmpegReader(fpath)
    frames = []
    for f in tqdm.tqdm_notebook(
        reader.nextFrame(), 
        total=reader.inputframenum,
        desc='load %s' % videoPrefix,
    ):
        frames.append(f)
    allFrames[videoPrefix] = frames

In [ ]:
def show(img, ax, title=None, doLegend=True):
    ax.imshow(img)
    hi = img.shape[0]
    
    mono = len(img.shape) == 2
    
    if not len(set(img.ravel())) == 2:
        def hist(ichannel, color, heightFraction=.25, histAlpha=.5):
            if not mono:
                histAlpha /= 2
            data = np.copy(img.reshape((img.shape[0], img.shape[1], -1))[:, :, ichannel]).ravel()
            hist, bins = np.histogram(data, bins=128)
            bins = (bins[:-1] + bins[1:]) / 2.
            imax = np.argmax(hist)
            vmax = bins[imax]
            
            brightest = max(bins[hist/max(hist)>1e-4])
            
            hist = np.log10(hist + 1e-10)
            hist -= hist.min()
            hist = hist * img.shape[0] / hist.max() * heightFraction
            bins /= 255
            bins *= img.shape[1]
            ax.fill_between(bins, hi, hi-hist, alpha=histAlpha, color=color, zorder=999)
            ax.axvline(bins[imax], label='mode: %d' % vmax, color=color)
            if mono:
                
                ax.axvline(
                    brightest / 255 * img.shape[1], 
                    label='~max: %d' % brightest, color=color, linestyle='--'
                )
            
        if mono:
            hist(0, 'white')
        else:
            for i in range(3):
                hist(i, ['red', 'blue', 'green'][i])
        if doLegend: ax.legend(fontsize=6, loc='upper right')
    ax.set_xlim(0, img.shape[1])
    ax.set_ylim(hi, -1) 
    
    ax.set_title(title)
    return fig, ax

In [ ]:
undistort = laneFindingPipeline.Undistorter()
undistort.fit()

In [ ]:
def plot(fig, axes, frame):
    colorFilter = laneFindingPipeline.ColorFilter()
    perspective = laneFindingPipeline.PerspectiveTransformer()
    
    fx = lambda bw: np.abs(cv2.Sobel(bw, cv2.CV_64F, 1, 0))
    fxd = colorFilter.dilateSobel

    img = perspective(undistort(frame))
    img = cv2.GaussianBlur(img, (5, 5), 0)
    eq = colorTools.equalizeHist(img)

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h_channel = hls[:, :, 0]
    l_channel = hls[:, :, 1]
    s_channel = hls[:, :, 2]
    
    hls = cv2.cvtColor(eq, cv2.COLOR_RGB2HLS)
    lq_channel = hls[:, :, 1]
    sq_channel = hls[:, :, 2]
    
    lab = cv2.cvtColor(eq, cv2.COLOR_RGB2LAB)
    labl_channel = lab[:, :, 0]
    labb_channel = lab[:, :, 2]
    
    luv = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    luvl_channel = luv[:, :, 0]
    
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hsvs_channel = hsv[:, :, 1]
    hsvh_channel = hsv[:, :, 0]

    # Erode a mask on light areas. This provides a mask
    # to exclude edges due to shadow.
    shadowMask = cv2.erode((gray > 16).astype('uint8'), colorTools.circleKernel(10), iterations=15)

#     grayFeatures = morphologicalSmoothing(shadowMask & fxd(gray))
#     bfeatures = fxd(b_channel)

    st = 150
    lt = 200
    
    r = 0
    c = 0
    ax = lambda : axes[r][c]
    
    show(frame, ax(), title='original frame'); c+= 1
    show(img, ax(), title='color'); c+= 1
    show(eq, ax(), title='eq(color)'); c+= 1
    show(gray, ax(), title='gray'); c+= 1
    #show(fx(gray), ax=axes[0][3], title='sobelx(gray)')
        
    c=0; r+= 1
    
    #show(h_channel, ax(), title='HLS hue'); c+= 1
    show(s_channel, ax(), title='HLS saturation'); c+= 1
    show(l_channel, ax(), title='HLS luminance'); c+= 1
    show(sq_channel, ax(), title='HLS (eq) saturation'); c+= 1
    #show(hsvh_channel, ax(), title='HSV h'); c+= 1
    #show(lq_channel, ax(), title='HLS (eq) luminance'); c+= 1
    #show(sobeltheta, ax(), title=r'sobel$_\theta$(LUV l)'); c+= 1
    #show(sobelx, ax(), title=r'sobel$_x$(LUV l)'); c+= 1
    #show(sobely, ax(), title=r'sobel$_y$(LUV l)'); c+= 1
    
    toSobel = (s_channel>st).astype('uint8')*255
    sobelx = colorTools.dilate(cv2.Sobel(toSobel, cv2.CV_64F, 1, 0), iterations=4)
    sobely = colorTools.dilate(cv2.Sobel(toSobel, cv2.CV_64F, 0, 1), iterations=4)
    sobelmag = np.log10(np.sqrt(sobelx ** 2 + sobely ** 2) + 1)
    sobeltheta = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    one = np.ones_like(sobeltheta, 'uint8')*240
    hue = colorTools.uint8scale(sobeltheta)
    mag = colorTools.uint8scale(sobelmag, lo=0)
    sobelhsv = np.dstack((hue, one, mag))
    show(
        cv2.GaussianBlur(
            cv2.cvtColor(sobelhsv, cv2.COLOR_HSV2RGB),
            (15, 15), 0
        ), 
        ax(), title=r'sobel$_\theta$(LUV l)', doLegend=False
    ); c+= 1
    
    c=0; r+= 1
    
    show(hsvs_channel, ax(), title='HSV s'); c+= 1
    show(luvl_channel, ax(), title='LUV l'); c+= 1
    show(labl_channel, ax(), title='LAB l'); c+= 1
    show(labb_channel, ax(), title='LAB B'); c+= 1
    
    c=0; r+= 1

    
    
    show(shadowMask, ax(), title='eroded non-shadow mask'); c+= 1
    #show(bfeatures, ax=axes[1][1], title='b features')
    #show(grayFeatures, ax=axes[1][2], title='gray features')
    #show(fxd(gray), ax=axes[1][3], title='sobelxDilate(gray)')
    show(fxd(luvl_channel), ax(), title='fxd(LUV l)'); c+= 1
    show(s_channel>st, ax(), title='HLS saturation > %s' % st); c+= 1
    show(l_channel>lt, ax(), title='HLS luminance > %s' % lt); c+= 1
    
    for r in axes:
        for ax in r:
            ax.title.set_fontsize(10)
            ax.set_xticks([])
            ax.set_yticks([])

    fig.subplots_adjust(bottom=0, left=0, right=1, top=.95, wspace=0.01, hspace=.1)

In [ ]:
%%time
fa = lambda : plt.subplots(nrows=4, ncols=4)
fig, axes = fa()
plot(fig, axes, allFrames['harder_challenge'][42])
fig.savefig('doc/grid.png')

In [ ]:
maxframes = 320000
overallBar = tqdm.tqdm_notebook(
    total=sum([min(len(fr), maxframes) for fr in allFrames.values()]), 
    unit='frame', desc='%d videos' % len(allFrames)
)

for videoPrefix in videoPrefices:
    frames = allFrames[videoPrefix][:maxframes]
    
    def yieldPlots():

        fig, axes = fa()

        for frame in frames:
            overallBar.update()

            plot(fig, axes, frame)

            yield utils.fig2img(fig)

            for r in axes:
                for ax in r:
                    ax.cla()
                    
        fig.clf()

    utils.saveVideo(
        yieldPlots(), 'filterOptions-%s-short.mp4' % videoPrefix, 
        total=len(frames), desc=videoPrefix,
    )